In [1]:
import datetime
from deta import Deta
import pandas as pd

# pertama
# DETA_KEY = "c0gxjy11k58_1LNFR6YDq8M1Ptm267jymu1Eau95ELjc"
# kedua
DETA_KEY = "c06jtxtz2ux_7jT2qFf4QXAwpfTSwDqzfekgEv1S7JSN"



deta = Deta(DETA_KEY)
# pertama
# db_transaksi = deta.Base("coba")
# db_user = deta.Base("user")

# kedua
db_transaksi = deta.Base("dataTransaksi")
db_data_masker = deta.Base("dataMasker")
db_hasil = deta.Base("dataHasil")



In [5]:
def cekHasil(period):
    existing_data = db_hasil.fetch({"tanggal_upload": period}).items

    if existing_data:
        return existing_data
    else:
        return 0


print(cekHasil("2023-04-30 21:01:46"))


0


In [ ]:
def insert_period(tanggal, merek, banyak, tanggal_upload):
    """Returns the report on a successful creation, otherwise raises an error"""
    return db_transaksi.put({"tanggal": tanggal, "merek": merek, "banyak": banyak, "tanggal_upload": tanggal_upload})


In [ ]:
def fetch_all_tanggal_upload():
    """Returns a list of all unique values in the tanggal_upload column"""
    all_data = db_transaksi.fetch().items
    unique_tanggal_upload = list(
        set([item['tanggal_upload'] for item in all_data]))
    return unique_tanggal_upload

print(fetch_all_tanggal_upload())

In [ ]:
# import itertools

def fetch_periods_by_date(tanggal_upload):
    """Returns a list of all periods with the specified upload date"""
    all_data = db_transaksi.fetch().items
    filtered_data = [
        item for item in all_data if item['tanggal_upload'] == tanggal_upload]
    return filtered_data

#     all_data = db_transaksi.fetch().items
#     filtered_data = [
#         item for item in all_data if item['tanggal_upload'] == tanggal_upload]

#     # group the filtered data by 'merek'
#     grouped_data = itertools.groupby(filtered_data, key=lambda x: x['merek'])

#     # calculate the total of 'banyak' for each group
#     result = []
#     for merek, group in grouped_data:
#         total_banyak = sum(item['banyak'] for item in group)
#         result.append({'merek': merek, 'total_banyak': total_banyak})

#     return result


    # all_data = db_transaksi.fetch().items
    # filtered_data = [
    #     item for item in all_data if item['tanggal_upload'] == tanggal_upload]

    # result = []
    # for item in filtered_data:
    #     added = False
    #     for res in result:
    #         if res['merek'] == item['merek']:
    #             res['total_banyak'] += item['banyak']
    #             added = True
    #             break
    #     if not added:
    #         result.append(
    #             {'merek': item['merek'], 'total_banyak': item['banyak']})

    # return result


print(fetch_periods_by_date("2023-04-29 13:46:56"))


In [ ]:
def merge_data(tanggal_upload):
    # Fetch data from both tables
    data_masker = db_data_masker.fetch().items
    data_transaksi = db_transaksi.fetch().items

    # Get total banyak for each mask brand from transaksi table
    total_banyak_by_merek = {}
    for item in data_transaksi:
        if item['tanggal_upload'] == tanggal_upload:
            if item['merek'] in total_banyak_by_merek:
                total_banyak_by_merek[item['merek']] += item['banyak']
            else:
                total_banyak_by_merek[item['merek']] = item['banyak']

    print (total_banyak_by_merek)   
    # Merge data from both tables by mask brand
    result = []
    for item_masker in data_masker:
        merek = item_masker['merek']
        stok = item_masker['stok_awal'] - total_banyak_by_merek.get(merek, 0)
        total_penjualan = total_banyak_by_merek.get(merek, 0)
        keuntungan = total_banyak_by_merek.get(
            merek, 0) * item_masker['keuntungan']

        result.append({
            'merek': merek,
            'stok': stok,
            'total_penjualan': total_penjualan,
            'keuntungan': keuntungan
        })

    return result

print(merge_data("2023-04-30 21:01:45"))


In [ ]:
def delete_data_by_date(tanggal_upload):
    """Deletes all data with the specified tanggal_upload"""
    data = db_transaksi.fetch({"tanggal_upload": tanggal_upload})
    for item in data.items:
        db_transaksi.delete(item["key"])


In [ ]:
def fetch_data_masker():
    data = db_data_masker.fetch().items
    print(data)

fetch_data_masker()


In [ ]:
def insert_data(merek, stok_awal, harga_awal, keuntungan):
    """Returns the report on a successful creation, otherwise raises an error"""
    return db_data_masker.put({
        'merek': merek,
        'stok_awal': stok_awal,
        'harga_awal': harga_awal,
        'keuntungan': keuntungan,
        'harga_jual': harga_awal + keuntungan
    })


def edit_data(key, merek, stok_awal, harga_awal, keuntungan, harga_jual):
    return db_data_masker.update({
        'merek': merek,
        'stok_awal': stok_awal,
        'harga_awal': harga_awal,
        'keuntungan': keuntungan,
        'harga_jual': harga_jual
    }, key)


def delete_data(key):
    return db_data_masker.delete(key)


In [ ]:
merek_list = [item.get('merek') for item in fetch_data_masker()]


def fetch_data_masker_by_merek(selected_merek):
    """Returns a list of all periods with the specified upload date"""
    all_data = db_data_masker.fetch().items
    filtered_data = [
        item for item in all_data if item['merek'] == selected_merek]
    return filtered_data


print(merek_list)
